In [138]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from numpy.linalg import inv
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor

In [2]:
df = pd.read_csv('/Users/herrakaava/Documents/ML_DATA/Advertising.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [4]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [5]:
X = df.drop('sales', axis=1)
y = df['sales']

<h3>Finding the parameter vector $\, \boldsymbol{\theta} \,$ with sklearn</h3>

In [6]:
pipe = make_pipeline(StandardScaler(),
                     SGDRegressor(max_iter=10000, tol=1e-5, penalty=None, eta0=0.01, n_iter_no_change=100))
pipe.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdregressor',
                 SGDRegressor(max_iter=10000, n_iter_no_change=100,
                              penalty=None, tol=1e-05))])

In [7]:
lin_reg = pipe.named_steps['sgdregressor']

In [16]:
pd.DataFrame({
    'coef': np.concatenate([lin_reg.intercept_, lin_reg.coef_], axis=0)
}, index=['intercept'] + X.columns.tolist())

,coef
intercept,14.022011
TV,3.920112
radio,2.791179
newspaper,-0.022483


<h3>Finding the parameter vector $\, \boldsymbol{\theta} \,$ with Stochastic Gradient Descent</h3>

Unlike batch gradient descent, which uses the whole training set to compute the gradients at every step (iteration), *stochastic gradient descent* picks a random data point in the training set at every step an computes the gradients only on that single data point.

With linear regression, the objective function one aims to minimize (usually) is

$$ \mathcal{J}(\boldsymbol{\theta}) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 = \frac{1}{n} (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta})^T \, (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta}). $$

To minimize the objective function, one needs to find the gradient of the objective function $\, \mathcal{J}(\boldsymbol{\theta}) \,$ with respect to the parameter vector $\, \boldsymbol{\theta}. \,$

$$ \frac{\partial}{\partial \boldsymbol{\theta}} \mathcal{J}(\boldsymbol{\theta}) = \begin{bmatrix}
\frac{\partial}{\partial \boldsymbol{\theta}_0} \mathcal{J}(\boldsymbol{\theta}) \\
\frac{\partial}{\partial \boldsymbol{\theta}_1} \mathcal{J}(\boldsymbol{\theta}) \\
\vdots \\
\frac{\partial}{\partial \boldsymbol{\theta}_p} \mathcal{J}(\boldsymbol{\theta}) \\
\end{bmatrix}. $$

The derivation is given below:

\begin{align*}
    \frac{1}{n} (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta})^T \, (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta}) &= \frac{1}{n} (\boldsymbol{y}^T - \boldsymbol{\beta}^T \boldsymbol{X}^T) \, (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta}) \\
    &= \frac{1}{n} (\boldsymbol{y}^T \boldsymbol{y} - \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\beta} - \boldsymbol{\beta}^T \boldsymbol{X}^T \boldsymbol{y} + \boldsymbol{\beta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\beta}) \\
    &= \frac{1}{n} (\boldsymbol{y}^T \boldsymbol{y} - \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\beta} - \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\beta} + \boldsymbol{\beta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\beta}) \\
    &= \frac{1}{n} (\boldsymbol{y}^T \boldsymbol{y} - 2 \boldsymbol{y}^T \boldsymbol{X} \boldsymbol{\beta} + \boldsymbol{\beta}^T \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\beta})
\end{align*}

Some derivation rules:

$$ \frac{\partial}{\partial \boldsymbol{x}} \boldsymbol{x}^T \boldsymbol{a} = \frac{\partial}{\partial \boldsymbol{x}} \boldsymbol{a}^T \boldsymbol{x} = \boldsymbol{a}. $$

$$ \frac{\partial}{\partial \boldsymbol{x}} \boldsymbol{x}^T \boldsymbol{A} \boldsymbol{x} = (\boldsymbol{A} + \boldsymbol{A}^T) \boldsymbol{x} = 2 \boldsymbol{A} \boldsymbol{x} \quad \text{(if} \, \boldsymbol{A} = \boldsymbol{A}^T). $$

Hence

\begin{equation*}
    \frac{\partial}{\partial \boldsymbol{\theta}} \mathcal{J}(\boldsymbol{\theta}) = \frac{2}{n} \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} - \frac{2}{n} \boldsymbol{X}^T \boldsymbol{y}
\end{equation*}

And the least squares solution is:

\begin{align*}
    \frac{2}{n} \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} - \frac{2}{n} \boldsymbol{X}^T \boldsymbol{y} &= 0 \\
    \frac{2}{n} \boldsymbol{X}^T \boldsymbol{X} \boldsymbol{\theta} &= \frac{2}{n} \boldsymbol{X}^T \boldsymbol{y}  \\
    \boldsymbol{\theta} &= (\boldsymbol{X}^T \boldsymbol{X})^{-1} \boldsymbol{X}^T \boldsymbol{y}.
\end{align*}

We will be using the gradient of the objective function with respect to  in the stochastic gradient descent optimization algorithm. Note that with SGD, since we use only one data point for the gradient calculations, we don't divide by n.

In [41]:
def scale_features(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled

In [34]:
def learning_rate_schedule(t, t0=5, t1=50):
    return t0 / (t + t1)

In [129]:
def SGD(X, y, n_epochs):
    # Randomly initialize the model parameters
    theta = np.random.normal(loc=0.0, scale=1.0, size=(4,1))    # Col vector
    
    # The number of training examples
    n = X.shape[0]
    
    # Stochastic gradient descent
    for epoch in range(n_epochs):    # Iterate through n epochs
        for iteration in range(n):   # Iterate through the training examples
            random_index = np.random.randint(low=0, high=n)
            x_i = X[random_index, :][np.newaxis, :]             # Row vector
            y_i = np.array([y[random_index]])[:, np.newaxis]      # 1x1
            grads = 2 * x_i.T @ x_i @ theta - 2 * x_i.T @ y_i
            alpha = learning_rate_schedule(t=epoch * n + iteration)
            theta -= alpha * grads
    
    return theta

In [130]:
# Scale the features
X_scaled = scale_features(X)

# Add a column of constants (for intercept calculation)
X_scaled = sm.add_constant(X_scaled)

In [131]:
theta = SGD(X_scaled, y.values, n_epochs=50)

In [136]:
pd.DataFrame({
    'theta': np.round(theta, 4).ravel()
}, index=['intercept'] + X.columns.tolist())

,theta
intercept,14.0535
TV,3.9036
radio,2.8161
newspaper,-0.0117


This solution of SGD is not that far away from the OLS solution, as can be seen below.

In [141]:
inv(X_scaled.T @ X_scaled) @ X_scaled.T @ y

array([14.0225    ,  3.91925365,  2.79206274, -0.02253861])